In [2]:
import time

from tqdm import tqdm
import numpy as np

In [3]:
## prepare blocks.
blocks = []
#blocks.append(np.zeros((4, 4)).astype(int))

blocks_ = np.zeros((4, 4)).astype(int)
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks_[3, 2] = 1
blocks_[3, 3] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0]) 
blocks_ = np.zeros((4, 3)).astype(int)
blocks_[1, 1] = 1
blocks_[2, 0] = 1
blocks_[2, 1] = 1
blocks_[2, 2] = 1
blocks_[3, 1] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0])
blocks_ = np.zeros((4, 3)).astype(int)
blocks_[1, 2] = 1
blocks_[2, 2] = 1
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks_[3, 2] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0]) 
blocks_ = np.zeros((4, 1)).astype(int)
blocks_[0, 0] = 1
blocks_[1, 0] = 1
blocks_[2, 0] = 1
blocks_[3, 0] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0])
blocks_ = np.zeros((4, 2)).astype(int)
blocks_[2, 0] = 1
blocks_[2, 1] = 1
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks.append(blocks_)

blocks

[array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 1, 1]]),
 array([[0, 0, 0],
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]]),
 array([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]]),
 array([[1],
        [1],
        [1],
        [1]]),
 array([[0, 0],
        [0, 0],
        [1, 1],
        [1, 1]])]

In [4]:
def get_n_col(pattern):
    return np.shape(blocks[pattern])[1]

    
def set_block(field, pattern, n_row, n_col, k):
    '''
    add the pattern*k to the field.
    (x, y) is the left-bottom position of the block.
    '''
    field_ = np.copy(field)
    block = blocks[pattern]
    for r in range(n_row, n_row+4):
        for c in range(n_col, n_col+get_n_col(pattern)):
            field_[r][c] += block[r-n_row][c-n_col] * k
    return field_


def move_block(field, pattern, n_row, n_col, jet):
    if jet == '<':
        if movable(field, pattern, n_row, n_col-1, n_row, n_col):
            field = set_block(field, pattern, n_row, n_col, -1)
            n_col -= 1
            field = set_block(field, pattern, n_row, n_col, 1)
    elif jet == '>':
        if movable(field, pattern, n_row, n_col+1, n_row, n_col):
            field = set_block(field, pattern, n_row, n_col, -1)
            n_col += 1
            field = set_block(field, pattern, n_row, n_col, 1)
            
    return field, n_col


def movable(field, pattern, n_row_next, n_col_next, n_row, n_col):
    if all([n_row_next >= 0, n_row_next < np.shape(field)[0]-3, n_col_next >=0, n_col_next < 7-get_n_col(pattern)+1]):
        field_ = set_block(field, pattern, n_row, n_col, -1)
        field_ = set_block(field_, pattern, n_row_next, n_col_next, 1)
        if np.max(field_) > 1:
            is_movable = False
        else:
            is_movable = True
    else:
        is_movable = False
    return is_movable


def drop_block(field, pattern, n_row, n_col):
    field = set_block(field, pattern, n_row, n_col, -1)
    n_row += 1
    field = set_block(field, pattern, n_row, n_col, 1)
    return field, n_row


def rest_block(field, pattern, jets, n_jet):
    n_row = 0
    n_col = 2    
    field = set_block(field, pattern, n_row, n_col, 1)
    #print(field)
    while(1):
        n_jet = n_jet % len(jets)
        field, n_col = move_block(field, pattern, n_row, n_col, jets[n_jet])
        n_jet += 1
        #print(field)
        #print(f"move {n_row}, {n_col}")
        if movable(field, pattern, n_row+1, n_col, n_row, n_col):
            field, n_row = drop_block(field, pattern, n_row, n_col)
            #print(field)
            #print(f"down {n_row}, {n_col}")
        else:
            break
    return field, n_jet


def get_tower_height(field):
    # get height of the tower.
    n_row_max, n_col_max = np.shape(field)
    nonzero = []
    # check the height of each column.
    for i in range(n_col_max):
        try: 
            x = list(field[:, i]!=0).index(1)
        except ValueError:
            x = n_row_max
        nonzero.append(x)

    block_top    = np.min(nonzero)
    block_bottom = np.max(nonzero)
    #print(f"{n_row_max}: {nonzero} >> {block_bottom}")
    
    return block_top, block_bottom


def update_field(field):
    block_top, block_bottom = get_tower_height(field)
    #print(f"block_bottom: {block_bottom}")
    n_rows_to_add = 7 - block_top
    filler = np.zeros((n_rows_to_add, n_col_max)).astype(int)
    field_ = np.vstack((filler, field[:block_bottom, :]))
#     # values > 1 are replaced with 1 for movable().
#     # because movable returns "false" if there are values > 1. 
#     n_rows, n_cols = np.where(field_>1)
#     if len(n_rows)>0:
#         for n_row, n_col in zip(n_rows, n_cols):
#             field_[n_row, n_col] = 1

    return field_, np.shape(field)[0] - block_bottom

In [ ]:
jets = list('>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>')
#with open('input.txt') as f:
#    jets = f.read().strip()

#n_blocks_max = 2022
n_blocks_max = 1000000000000

n_col_max = 7
n_row_max = 7
pattern = 0
n_jet = 0
block_removed = 0
field = np.zeros((n_row_max, n_col_max)).astype(int) 
time_start = time.time()
#for n_block in range(0, n_blocks_max)
for n_block in tqdm(range(0, n_blocks_max)):
    #print("\r"+str(n_block),　end="")
    if n_block > 0:
        field, block_removed_ = update_field(field)
        block_removed += block_removed_
        n_row_max = np.shape(field)[1]
        #pattern = (pattern+1) % 5
        pattern = n_block % 5
        
    field, n_jet = rest_block(field, pattern, jets, n_jet)
    #print(n_block+1)
    #print(field)
#     if n_block > 1:
#         print(f"block_removed: {block_removed_}")

block_top, _ = get_tower_height(field)
# print(np.shape(field)[0])
# print(block_top)
# print(block_removed)

tower_height = (np.shape(field)[0] - block_top) + block_removed
print(f"tower height: {tower_height}")
# print(field)
print(f"elapsed time: {time.time()-time_start}[s]")

  0%|                                                                                               | 295795/1000000000000 [03:44<255785:22:18, 1085.98it/s]